In [1]:
#DATA EXTRACTION
import openpyxl
from bs4 import BeautifulSoup
from urllib.request import urlopen, HTTPError
import ssl
import gzip
#Ignore ssl certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE0

workbook = openpyxl.load_workbook("Input.xlsx")
sheet = workbook["Sheet1"]
#Accessing each rows
for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=1, max_col=2):
    url_id_cell, url_cell = row
    url_id = url_id_cell.value
    url = url_cell.value
    try:
        with urlopen(url) as response:
            if response.info().get('Content-Encoding') == 'gzip':
                with gzip.GzipFile(fileobj=response) as gzipped_response:
                    html_content = gzipped_response.read().decode('utf-8')
            else:
                html_content = response.read().decode('utf-8')

    except HTTPError as e:
        if e.code == 404:
            print(f"Error 404: {url} for {url_id} not found. Skipping...")
        continue
    soup = BeautifulSoup(html_content, "html.parser")
    #print(soup.prettify())
    head_tag = soup.head
    title_tag = head_tag.title
    try:
        title_text = title_tag.text
    except:
        title_text = "No Title"
        print(f"No title for {url_id}")
        continue
    article_div = soup.find("div", class_="td-post-content tagdiv-type")
    if article_div:
        content = '\n'.join(paragraph.text for paragraph in article_div.find_all('p'))
    with open(f"{url_id}.txt", "w", encoding="utf-8") as file:
        file.write(f"{title_text}\n")
        #file.write("\n")
        file.write(content)


Error 404: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/ for 11668.0 not found. Skipping...
Error 404: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/ for 17671.4 not found. Skipping...


In [2]:
import os
#Cleaning using stop words
stop_words_list = []
for filename in os.listdir("StopWords"):
    file_path = os.path.join("StopWords", filename)
    with open(file_path, "r")  as file:
        for word in file:
            stop_words_list.extend(word.strip().split())
for item in stop_words_list:
    if item == "|":
        stop_words_list.remove(item)
master_dict_cleaned = {"positive_words" : [], "negative_words":[]}
for file_desig in os.listdir("MasterDictionary"):
    path_file = os.path.join("MasterDictionary", file_desig)
    if file_desig == "negative-words.txt":
        with open(path_file, "r") as files:
            for word in files:
                if word.strip() in stop_words_list:
                    continue
                else:
                    master_dict_cleaned["negative_words"].append(word.strip())
    elif file_desig == "positive-words.txt":
        with open(path_file, "r") as files:
            for word in files:
                if word.strip() in stop_words_list:
                    continue
                else:
                    master_dict_cleaned["positive_words"].append(word.strip())

In [46]:
# Full text analysis
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import cmudict
from nltk.corpus import stopwords
import re
import openpyxl
import string
import pandas as pd
nltk.download("punkt")
nltk.download("cmudict")
nltk.download('stopwords')

def count_syllables(word):
    try:
        return max([len(list(y for y in x if y[-1].isdigit())) for x in cmudict.dict()[word.lower()]])
    except KeyError:
        # If the word is not in the CMU Pronouncing Dictionary, return 0
        return 0
def find_complex_words(text):
    complex_words = [word for word in text if count_syllables(word) >= 2]
    return complex_words
def count_syllables(word):
    # Count the number of vowels in a word
    vowels = "aeiouy"
    count = 0
    # Handle exceptions for words ending with "es" and "ed"
    if word.endswith(("es", "ed")):
        return count
    for char in word:
        if char.lower() in vowels:
            count += 1
    return count
def count_syllables_in_text(text):
    # Count syllables for each word and sum them up
    total_syllables = sum(count_syllables(word) for word in text)
    return total_syllables
       
def count_personal_pronouns(words):
    # Define a regex pattern for personal pronouns
    pronoun_pattern = r'\b(?:I|we|my|ours|us)\b'

    # Exclude instances where "US" refers to the country
    country_pattern = r'\bUS\b'

    # Join the list of words into a string for regex matching
    text = ' '.join(words)

    # Use regex to find matches for personal pronouns
    pronoun_matches = re.findall(pronoun_pattern, text, flags=re.IGNORECASE)

    # Use regex to find matches for the country name "US"
    country_matches = re.findall(country_pattern, text)

    # Exclude instances of "US" referring to the country from personal pronoun matches
    filtered_pronoun_matches = [pronoun.lower() for pronoun in pronoun_matches if pronoun.lower() not in country_matches]

    # Count the number of personal pronoun matches
    count = len(filtered_pronoun_matches)

    return count
#progress_report = 1
output_df = pd.read_excel('Output Data Structure.xlsx')
for filename in os.listdir("text_files"):
    file_path = os.path.join("text_files", filename)
    with open(file_path, "r", encoding= "utf-8") as file_hnd:
        text = file_hnd.read()
        #Using nltk tokenization as required in Text Analysis.docx
        tokens = word_tokenize(text)
        positive_score = 0
        negative_score = 0
        #1: SENTIMENTAL ANALYSIS
        cleaned_words = [word for word in tokens if word.isalpha() and word not in stop_words_list]
        for item in cleaned_words:
            if item in master_dict_cleaned["positive_words"] :
                positive_score+=1
            elif item in master_dict_cleaned["negative_words"] :
                negative_score+=1
            else:
                continue
        polarity_score = (positive_score - negative_score)/ ((positive_score + negative_score) + 0.000001)
        subjectivity_score = (positive_score + negative_score)/ ((len(cleaned_words)) + 0.000001)
        #2: ANALYSIS OF READABILITY
        sentences = text.split(".")
        wds = text.split()
        avg_sentence_length = len(wds)/ len(sentences)
        complx_words = find_complex_words(wds)
        percentage_complx_word = len(complx_words)/len(wds)
        fog_index = 0.4 * (avg_sentence_length + percentage_complx_word)
        #3: Average Number of Words per sentence
        avg_words_sentence = len(wds)/ len(sentences)
        #4: Complex word count
        complx_words_count = len(complx_words)
        #5: Word count per the required nltk stopwords usage
        nltk_stop_words = set(stopwords.words('english'))
        nltk_cleaned_words = [word for word in tokens if word.lower() not in nltk_stop_words]
        word_count = len(nltk_cleaned_words)
        #6: Syllable count per word
        syllable_per_word = (count_syllables_in_text(wds))/ len(wds)
        #7: Personal Pronouns
        personal_prons = count_personal_pronouns(wds)
        #8: Average word length
        total_characters = sum(len(list(wd)) for wd in wds)
        #print(total_characters, len(wds))
        avg_word_length = total_characters / len(wds)
        #Create a dictionary of the things we want
        output_data = {"POSITIVE SCORE": positive_score, "NEGATIVE SCORE": negative_score, "POLARITY SCORE": polarity_score, 
                       "SUBJECTIVITY SCORE": subjectivity_score, "AVG SENTENCE LENGTH": avg_sentence_length, 
                       "PERCENTAGE OF COMPLEX WORDS": percentage_complx_word, "FOG INDEX": fog_index, 
                       "AVG NUMBER OF WORDS PER SENTENCE": avg_words_sentence, "COMPLEX WORD COUNT": complx_words_count, 
                       "WORD COUNT": word_count, "SYLLABLE PER WORD": syllable_per_word, "PERSONAL PRONOUNS": personal_prons,
                       "AVG WORD LENGTH": avg_word_length}
        
        url_id = float(filename.replace(".txt",""))
        filtered_df = output_df[~output_df["URL_ID"].isin([11668.0, 17671.4])]
        for i, row in filtered_df.iterrows():
            if url_id == row["URL_ID"]:
                # If the URL_ID matches, update the values in output_df for each key in output_data
                for key, value in output_data.items():
                    output_df.at[i, key] = value
    #print(progress_report)
    #progress_report +=1
output_df.to_excel("Output Data Structure.xlsx", index=False)
#pd.DataFrame(output_df)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\idakw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\idakw\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\idakw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
df = output_df[output_df["URL_ID"] == 11668.0]
df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
24,11668.0,https://insights.blackcoffer.com/how-neural-ne...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
